## Create training datasets for decoder

In [ ]:
import sys
sys.path.append('../')
import utils.mat_to_tfrecord
from pathlib import Path
import numpy as np
from T18_SessionArgsArraybyArray import get_session_info

sessions = [
   't18.2024.12.04',
   't18.2024.12.05',
   't18.2025.01.14',
   't18.2025.01.15',
   't18.2025.01.21',
   't18.2025.01.22',
   't18.2025.02.04',
   't18.2025.02.05',
     ]

participant = 't18'
bin_compression_factor = 2
channels_to_exclude = list(range(0,0)) 
channels_to_zero = list(range(0,0)) #[] # leave empty to not zero anything

for session in sessions:
    
    trials_to_remove, block_nums, num_test_trials = get_session_info(session)
    session_path = str(Path('../../Data', participant, session, 'Typing'))
    tfdata_path = str(Path(session_path, 'tfdata_20ms'))

    print(f'Sesison path: {session_path}')
    print(f'tfdata path: {session_path}')
    print('\n')

    args = {
        'session_mat_path': session_path,
        'block_nums': block_nums,
        'num_test_trials': num_test_trials,
        'trials_to_remove': trials_to_remove,
        'channels_to_exclude': channels_to_exclude,
        'channels_to_zero': channels_to_zero,
        'include_thresh_crossings': True,
        'include_spike_power': True,
        'spike_pow_max': 50000,
        'z_score_data': True,
        'global_std': True,
        'bin_compression_factor': bin_compression_factor,
        'save_path': tfdata_path,
    }

    utils.mat_to_tfrecord.main(args)

## Train the decoder. Remember to Restart the notebook first!

In [1]:
from omegaconf import OmegaConf
import sys
import numpy as np
sys.path.append('../')
import utils.brainToText_trainDecoder
import gc
all_channel_ranges = [(0,384), # All channels
                      (0, 64), (64, 128), 
                      (128, 192), (192, 256), (256, 320), (320, 384), # Each array
                      (256, 384)] # Each hemisphere

for channel_range in all_channel_ranges:
    importlib.reload(brainToText_trainDecoder)
    args = OmegaConf.load('T18_trainArgsArraybyArray.yaml')
    num_features = (channel_range[1] - channel_range[0]) * 2
    print(num_features)
    args['dataset']['nInputFeatures'] = num_features
    args['model']['inputLayerSize'] = num_features
    args['model']['inputNetwork']['inputLayerSizes'] = [num_features]
    out_dir = 'T18ArraybyArrayModels/' + str(channel_range[0]) + '-' + str(channel_range[1])
    args['outputDir'] = out_dir
    args['dataset']['use_channels'] = list(range(channel_range[0], channel_range[1])) + list(range(channel_range[0] + 384, channel_range[1] + 384))
    args['loadDir'] = 'null'
    decoder = utils.brainToText_trainDecoder.brainToText_decoder(args)
    infOut, stats = decoder.train()
    decoder = None
    args['loadDir'] = out_dir
    
    decoder = utils.brainToText_trainDecoder.brainToText_decoder(args)
    out, out_by_day = decoder.inference()
    print(out['seqErrorRateIndividual'])
    np.save('T18Train-' + str(channel_range[0]) + '-' + str(channel_range[1]) +'-RawCERs.npy',out['seqErrorRateIndividual'])
    decoder = None
    gc.collect()

2025-03-13 21:53:27.464924: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-13 21:53:27.487658: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-13 21:53:27.487676: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-13 21:53:27.487706: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-13 21:53:27.493027: I tensorflow/core/platform/cpu_feature_g

256
2025-03-13 21:53:28,550: Using GPU #: 0


2025-03-13 21:53:28.556281: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-03-13 21:53:28.603219: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-03-13 21:53:28.630640: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Model: "gru"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_1 (GRU)                 multiple                  6687744   
                                                                 
 gru_2 (GRU)                 multiple                  1575936   
                                                                 
 gru_3 (GRU)                 multiple                  1575936   
                                                                 
 gru_4 (GRU)                 multiple                  1575936   
                                                                 
 gru_5 (GRU)                 multiple                  1575936   
                                                                 
 dense (Dense)               multiple                  15903     
                                                                 
Total params: 13007903 (49.62 MB)
Trainable params: 13007903 (4

2025-03-13 21:53:36.420949: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2025-03-13 21:53:36.686271: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x789e563c54f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-03-13 21:53:36.686288: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2025-03-13 21:53:36.690423: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-03-13 21:53:36.748521: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2025-03-13 21:53:37,642: Train batch 0: loss: 652.16 gradNorm: 241.18 
(40, 2506, 256)
(10, 2041, 256)
2025-03-13 21:53:37,987: Val batch: CER (t18.2025.01.15): 0.955
2025-03-13 21:53:37,988: Val batch 0: CER (avg): 0.955 
2025-03-13 21:53:38,153: Model checkpoint saved to: /home/justin/Projects/NGEC/brand-modules/typing-east/T18ArraybyArray/256-384/
2025-03-13 21:53:38,157: Saved training args.yaml to: /home/justin/Projects/NGEC/brand-modules/typing-east/T18ArraybyArray/256-384/
2025-03-13 21:53:38,158: Batches since validation CER improved: 0
2025-03-13 21:53:38,565: Train batch 1: loss: 453.56 gradNorm: 207.28 
2025-03-13 21:53:39,084: Train batch 2: loss: 774.79 gradNorm: 498.93 
2025-03-13 21:53:39,309: Train batch 3: loss: 729.95 gradNorm: 578.95 
2025-03-13 21:53:40,616: Train batch 4: loss: 1511.95 gradNorm: 1700.73 
2025-03-13 21:53:41,537: Train batch 5: loss: 787.55 gradNorm: 1052.58 
2025-03-13 21:53:41,620: Train batch 6: loss: 311.75 gradNorm: 447.67 
2025-03-13 21:53:42,